# Control optimization in NV center (multiparameter) 

This is the source codes for the example discussed in Sec. IV F in Ref. [1].

The Hamiltonian of a controlled system can be written as
\begin{align}
H = H_0(\textbf{x})+\sum_{k=1}^K u_k(t) H_k,
\end{align}

where $H_0(\textbf{x})$ is the free evolution Hamiltonian with unknown parameters $\textbf{x}$ and $H_k$ 
represents the $k$th control Hamiltonian with $u_k$ the correspong control coefficient.

In the multiparameter scenario, the dynamics of electron and nuclear coupling in NV$^{-}$ can be expressed as
\begin{align}
\partial_t\rho=-i[H_0+H_{\mathrm{c}},\rho]+\frac{\gamma}{2}(S_3\rho S_3-S^2_3\rho-\rho S^2_3)
\end{align}

with $\gamma$ the dephasing rate. And
\begin{align}
H_0/\hbar=DS^2_3+g_{\mathrm{S}}\vec{B}\cdot\vec{S}+g_{\mathrm{I}}\vec{B}\cdot\vec{I}+\vec{S}^{\,\mathrm{T}}\mathcal{A}\vec{I}
\end{align}

is the free evolution Hamiltonian, where $\vec{S}=(S_1,S_2,S_3)^{\mathrm{T}}$ and $\vec{I}=(I_1,I_2,I_3)^{\mathrm{T}}$ with 
$S_i=s_i\otimes I$ and $I_i=I\otimes \sigma_i$ ($i=1,2,3$) the electron and nuclear operators. $\mathcal{A}=\mathrm{diag}
(A_1,A_1,A_2)$ is the hyperfine tensor with $A_1$ and $A_2$ the axial and transverse magnetic hyperfine coupling coefficients.
The coefficients $g_{\mathrm{S}}=g_\mathrm{e}\mu_\mathrm{B}/\hbar$ and $g_{\mathrm{I}}=g_\mathrm{n}\mu_\mathrm{n}/\hbar$, 
where $g_\mathrm{e}$ ($g_\mathrm{n}$) is the $g$ factor of the electron (nuclear), $\mu_\mathrm{B}$ ($\mu_\mathrm{n}$) is the Bohr (nuclear) magneton and $\hbar$ is the Plank's constant. $\vec{B}$ is the magnetic field which be estimated. The control Hamiltonian is
\begin{align}
H_{\mathrm{c}}/\hbar=\sum^3_{i=1}\Omega_i(t)S_i
\end{align}

with $\Omega_i(t)$ the time-dependent Rabi frequency.

In this case, the initial state is taken as $\frac{1}{\sqrt{2}}(|1\rangle+|\!-\!1\rangle)\otimes|\!\!\uparrow\rangle$, 
where $\frac{1}{\sqrt{2}}(|1\rangle+|\!-\!1\rangle)$ is an electron state with $|1\rangle$ $(|\!-\!1\rangle)$ the 
eigenstate of $s_3$ with respect to the eigenvalue $1$ ($-1$). $|\!\!\uparrow\rangle$ is a nuclear state and 
the eigenstate of $\sigma_3$ with respect to the eigenvalue 1. $W$ is set to be $I$.

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2022.xxxx.

In [ ]:
from quanestimation import *
import numpy as np

# initial state
rho0 = np.zeros((6,6), dtype=np.complex128)
rho0[0][0], rho0[0][4], rho0[4][0], rho0[4][4] = 0.5, 0.5, 0.5, 0.5
# free Hamiltonian
sx = np.array([[0., 1.],[1., 0.]])
sy = np.array([[0., -1.j],[1.j, 0.]]) 
sz = np.array([[1., 0.],[0., -1.]])
ide2 = np.array([[1., 0.],[0., 1.]])
s1 = np.array([[0., 1., 0.],[1., 0., 1.],[0., 1., 0.]])/np.sqrt(2)
s2 = np.array([[0., -1.j, 0.],[1.j, 0., -1.j],[0., 1.j, 0.]])/np.sqrt(2)
s3 = np.array([[1., 0., 0.],[0., 0., 0.],[0., 0., -1.]])
ide3 = np.array([[1., 0., 0.],[0., 1., 0.],[0., 0., 1.]])
I1, I2, I3 = np.kron(ide3, sx), np.kron(ide3, sy), np.kron(ide3, sz)
S1, S2, S3 = np.kron(s1, ide2), np.kron(s2, ide2), np.kron(s3, ide2)
B1, B2, B3 = 5.0e-4, 5.0e-4, 5.0e-4
cons = 100
D = (2*np.pi*2.87*1000)/cons
gS = (2*np.pi*28.03*1000)/cons
gI = (2*np.pi*4.32)/cons
A1 = (2*np.pi*3.65)/cons
A2 = (2*np.pi*3.03)/cons
H0 = D*np.kron(np.dot(s3, s3), ide2)+gS*(B1*S1+B2*S2+B3*S3)+gI*(B1*I1+B2*I2+B3*I3)+\
     + A1*(np.kron(s1, sx)+np.kron(s2, sy)) + A2*np.kron(s3, sz)
# derivatives of the free Hamiltonian on B1, B2 and B3
dH = [gS*S1+gI*I1, gS*S2+gI*I2, gS*S3+gI*I3]
# control Hamiltonians 
Hc = [S1, S2, S3]
# dissipation
decay = [[S3,2*np.pi/cons]]  
# measurement
M = []
for i in range(len(rho0)):
    M_tp = np.dot(basis(len(rho0), i), basis(len(rho0), i).conj().T)
    M.append(M_tp)
# time length for the evolution
tspan = np.linspace(0.0, 2.0, 4000)
# guessed control coefficients
cnum = 10
np.random.seed(1234)
ini_1 = np.zeros((len(Hc), cnum))
ini_2 = 0.2*np.ones((len(Hc), cnum))
ini_3 = -0.2*np.ones((len(Hc), cnum))
ini_4 = np.array([np.linspace(-0.2,0.2,cnum) for i in range(len(Hc))])
ini_5 = np.array([np.linspace(-0.2,0.0,cnum) for i in range(len(Hc))])
ini_6 = np.array([np.linspace(0,0.2,cnum) for i in range(len(Hc))])
ini_7 = -0.2*np.ones((len(Hc), cnum))+0.01*np.random.random((len(Hc), cnum))
ini_8 = -0.2*np.ones((len(Hc), cnum))+0.01*np.random.random((len(Hc), cnum))
ini_9 = -0.2*np.ones((len(Hc), cnum))+0.05*np.random.random((len(Hc), cnum))
ini_10 = -0.2*np.ones((len(Hc), cnum))+0.05*np.random.random((len(Hc), cnum))
ctrl0 = [ini_1, ini_2, ini_3, ini_4, ini_5, ini_6, ini_7, ini_8, ini_9, ini_10]

Control algorithm: auto-GRAPE

In [ ]:
GRAPE_paras = {"Adam":True, "ctrl0":[ini_1], "max_episode":300, "epsilon":0.01, "beta1":0.90, "beta2":0.99}
control = ControlOpt(savefile=False, method="auto-GRAPE", **GRAPE_paras)
control.dynamics(tspan, rho0, H0, dH, Hc, decay=decay, ctrl_bound=[-0.2,0.2])

In [ ]:
# choose QFIM as the objective function
control.QFIM()

In [ ]:
# choose CFIM as the objective function
control.CFIM(M)

Control algorithm: PSO

In [ ]:
PSO_paras = {"particle_num":10, "ctrl0":ctrl0, "max_episode":[1000,100], "c0":1.0, "c1":2.0, "c2":2.0, "seed":1234}
control = ControlOpt(savefile=False, method="PSO", **PSO_paras)
control.dynamics(tspan, rho0, H0, dH, Hc, decay=decay, ctrl_bound=[-0.2,0.2])

In [ ]:
# choose QFIM as the objective function
control.QFIM()

In [ ]:
# choose CFIM as the objective function
control.CFIM(M=M)

Control algorithm: DE

In [ ]:
DE_paras = {"popsize":10, "ctrl0":ctrl0, "max_episode":1000, "c":1.0, "cr":0.5, "seed":1234}
control = ControlOpt(savefile=False, method="DE", **DE_paras)
control.dynamics(tspan, rho0, H0, dH, Hc, decay=decay, ctrl_bound=[-0.2,0.2])

In [ ]:
# choose QFIM as the objective function
control.QFIM()

In [ ]:
# choose CFIM as the objective function
control.CFIM(M=M)

Control algorithm: DDPG

In [ ]:
DDPG_paras = {"layer_num":4, "layer_dim":250, "max_episode":500, "seed":1234}
control = ControlOpt(savefile=False, method="DDPG", **DDPG_paras)
control.dynamics(tspan, rho0, H0, dH, Hc, decay=decay, ctrl_bound=[-0.2,0.2])

In [ ]:
# choose QFIM as the objective function
control.QFIM()

In [ ]:
# choose CFIM as the objective function
control.CFIM(M=M)